**Problema 10**
------------------------------------------------------------------------------
***

> En este script se resuelve el P10 utilizando la elástica y superposición.

In [1]:
from sympy import *             # Librería para trabajo simbólico
import math                     # Librería para utilizar símbolos matemáticos como el número pi, que se escribe como math.pi
import numpy as np              # Librería para poder trabajar con matrices y vectores
import matplotlib.pyplot as plt # Librería para poder dibujar gráficas

x,a,P,Mo,Ra,Lt=symbols('x a P Mo Ra Lt') #variables simbólicas, donde Ra es la incógnita hiperestática y Lt la longitud total
R0y,R1y,R2y=symbols('R0y R1y R2y')
M0z,M1z,M2z=symbols('M0z M1z M2z')

A1,B1,A2,B2=symbols('A1 B1 A2 B2') #constantes de integración para la elástica

#solicitaciones
P0=Ra #N  la carga en x=0, como Ra es la incógnita hiperestática la ponemos como si fuera una carga conocida
P1=-P #N  la carga en x=L1

T1=-Mo #Nm  el par en x=L1

#geometría
L1=a #longitud tramo 1
L2=Lt-a #longitud tramo 2

L=L1+L2 #longitud total

#material
E=symbols('E')
I=symbols('I')


#Cálculo estático. Las ecuaciones ΣFy=0 y ΣMz=0 aparecen en la primera y segunda linea. Las reacciones que no participen
#en las ecuaciones (R0y,R1y,R2y,M0z,M1z,M2z) hay que añadirlas en las lineas de abajo
#Nota: Todas las reacciones y solicitaciones se consideran positivos en dirección positiva de los ejes cartesianos.
sol,=linsolve([  Ra+R2y+P1,   #se pone todo positivo siempre
               M0z +P1*L1+T1+R2y*(L1+L2),
              R0y,
              R1y,
              M1z,
              M2z,
             ], (R0y,R1y,R2y,M0z,M1z,M2z))
print ("la solución es: (R0y,R1y,R2y,M0z,M1z,M2z)=",sol)

#reacciones
R0y=sol[0] #N reacción en y en x=0 
R1y=sol[1] #N reacción en y en x=L1 
R2y=sol[2] #N reacción en y en x=L1+L2 
M0z=sol[3] #Nm reacción momento en x=0 
M1z=sol[4] #Nm reacción momento en x=L1 
M2z=sol[5] #Nm reacción momento en x=L1+L2 


#-------------------------------------------------------------------------------------#
#---------------------------------- CÁLCULO TRAMOS -----------------------------------#
#-------------------------------------------------------------------------------------#

#--------------------------------------TRAMO 1----------------------------------------#
#Definimos el cortante
VL=R0y+P0 #Este es el valor del cortante a la izquierda del tramo 1, lo que llamamos V+(x=0)
def cortante1(x):        
    return  VL 

#Definimos el flector
ML=-M0z #Este es el valor del flector a la izquierda del tramo 1, lo que llamamos M+(x=0)
def flector1(x):        
    return  integrate(cortante1(x), (x, 0, x)) + ML #en MPa 

V1=cortante1(x) 
M1=flector1(x)
print("La expresión del cortante en el tramo 1 es: ", V1)
print("La expresión del flector en el tramo 1 es: ", M1)


#Elástica
def giro1(x,A1):        
    return  integrate(flector1(x)/(E*I),x) + A1 #en MPa

def deflexion1(x,A1,B1):        
    return  integrate(giro1(x,A1),x) + B1 #en MPa

G1=giro1(x,A1) 
U1=deflexion1(x,A1,B1)
print("La expresión del giro en el tramo 1 es: ", G1)
print("La expresión de la deflexion en el tramo 1 es: ", U1)
print("")


#--------------------------------------TRAMO 2----------------------------------------#
#Definimos el cortante
VL=V1.subs([(x,L1)]) + R1y + P1 #Este es el valor del cortante a la izquierda del tramo 2, lo que llamamos V+(x=L1) que se 
                          #calcula mediante el equilibrio de la rebanada como el cortante a la izquierda V-(x=L1) más
                          #la carga puntual en x=L1
def cortante2(x):        
    return  VL 

#Definimos el flector
ML=M1.subs([(x,L1)]) - M1z - T1 #Este es el valor del flector a la izquierda, lo que llamamos M+(x=L1)
def flector2(x):        
    return  factor(integrate(cortante2(x), (x, L1, x)) + ML) #en MPa 

V2=cortante2(x) 
M2=flector2(x)
if L2 !=0:
    print("La expresión del cortante en el tramo 2 es: ", V2)
    print("La expresión del flector en el tramo 2 es: ", M2)

#elástica
def giro2(x):        
    return  simplify(integrate(flector2(x)/(E*I),x) + A2) #en MPa

def deflexion2(x):        
    return  simplify(integrate(giro2(x),x) + B2) #en MPa

G2=giro2(x) 
U2=deflexion2(x)
if L2 !=0:
    print("La expresión del giro en el tramo 2 es: ", G2)
    print("La expresión de la deflexion en el tramo 2 es: ", U2)
    print("",simplify(G2.subs([(x,Lt)])) )


    
#-------------------------------------------------------------------------------------#
#----------------------------------- C.C. ELÁSTICA -----------------------------------#
#-------------------------------------------------------------------------------------#      
G1_0=G1.subs([(x,0)])    #giro en tramo 1, punto 0
G1_1=G1.subs([(x,L1)])   #giro en tramo 1, punto 1
G2_1=G2.subs([(x,L1)])   #giro en tramo 2, punto 1
G2_2=G2.subs([(x,L1+L2)])#giro en tramo 2, punto 2 


U1_0=U1.subs([(x,0)])    #deflexion en tramo 1, punto 0
U1_1=U1.subs([(x,L1)])   #deflexion en tramo 1, punto 1
U2_1=U2.subs([(x,L1)])   #deflexion en tramo 2, punto 1
U2_2=U2.subs([(x,L1+L2)])#deflexion en tramo 2, punto 2 
 
#aquí con el comando "linsolve" resolvemos el sistema de 5 ecuaciones y 5 incógnitas para despejar Ra.
#Las incógnitas serán las 4 constantes de integración A1,B1,B2,A2 y Ra.
#Las ecuaciones vendrán dadas por las Condiciones de Contorno para la elástica:
# - como tenemos 2 tramos, impondremos que en la unión del tramo 1 y el 2 (en x=L1=a), la deflexión y el giro sean el mismo 
#   para ambos tramos: U1_1-U2_1=0 y G1_1-G2_1=0              ---> 2 ecuaciones
# - además, en los apoyos tendremos G1_0=0, U1_0=0 y U2_2=0   ---> 3 ecuaciones
sole,=linsolve([  G1_0,
                  U1_0,
                  U2_2,
                  G1_1-G2_1,
                  U1_1-U2_1
             ], (A1,B1,B2,A2,Ra))
#el vector "sole" guarda la solución para A1,B1,B2,A2,Ra

print("")
print("")
print("Ra=", factor(sole[4]))
print("")


la solución es: (R0y,R1y,R2y,M0z,M1z,M2z)= (0, 0, P - Ra, -Lt*(P - Ra) + Mo + P*a, 0, 0)
La expresión del cortante en el tramo 1 es:  Ra
La expresión del flector en el tramo 1 es:  Lt*(P - Ra) - Mo - P*a + Ra*x
La expresión del giro en el tramo 1 es:  A1 + Ra*x**2/(2*E*I) + x*(Lt*P - Lt*Ra - Mo - P*a)/(E*I)
La expresión de la deflexion en el tramo 1 es:  A1*x + B1 + Ra*x**3/(6*E*I) + x**2*(Lt*P - Lt*Ra - Mo - P*a)/(2*E*I)

La expresión del cortante en el tramo 2 es:  -P + Ra
La expresión del flector en el tramo 2 es:  -(-Lt + x)*(P - Ra)
La expresión del giro en el tramo 2 es:  (A2*E*I + Lt*x*(P - Ra) + x**2*(-P + Ra)/2)/(E*I)
La expresión de la deflexion en el tramo 2 es:  (E*I*(A2*x + B2) + Lt*x**2*(P - Ra)/2 + x**3*(-P + Ra)/6)/(E*I)
 (A2*E*I + Lt**2*(P - Ra)/2)/(E*I)


Ra= (2*Lt**3*P - 6*Lt*Mo*a - 3*Lt*P*a**2 + 3*Mo*a**2 + P*a**3)/(2*Lt**3)



In [21]:
#-------------------------------------------------------------------------------------#
#----------------------------- CÁLCULO POR SUPERPOSICIÓN -----------------------------#
#-------------------------------------------------------------------------------------#  

#para hacer el cálculo por superposición, considero Rb la incógnita hiperestática, es decir, supongo que esta será
#una solicitación y luego impondré que el desplazamiento vertical del punto B será cero
Rb=symbols('Rb') 

d1= -Rb*Lt**3/(3*E*I)      #deflexión para el caso 1
d2= P*a**2/(6*E*I)*(3*Lt-a)#deflexión para el caso 2
d7= Mo*a/(2*E*I)*(2*Lt-a)  #deflexión para el caso 7

#ahora impongo que el desplazamiento vertical de B, calculado como la suma de d1+d2+d7, es cero y resuelvo Rb
sole,=linsolve([  d1+d2+d7,
             ], (Rb))

print("Rb=", factor((sole[0])))

#y ahora calculo Ra utilizando ΣFy=0 ->  Ra+Rb-P=0,
print("Ra=", factor(P-(sole[0])))


Rb= -a*(-6*Lt*Mo - 3*Lt*P*a + 3*Mo*a + P*a**2)/(2*Lt**3)
Ra= (2*Lt**3*P - 6*Lt*Mo*a - 3*Lt*P*a**2 + 3*Mo*a**2 + P*a**3)/(2*Lt**3)
